In [13]:
import numpy as np
import pandas as pd
import dill as pkl
from collections import defaultdict

# Raw Data - will not use

In [32]:
data1 = pd.read_csv('genres.csv', header=None, encoding = "ISO-8859-1")
data2 = pd.read_csv('genres2.csv', header=None, encoding = "ISO-8859-1")
data3 = pd.read_csv('genres3.csv', header=None, encoding = "ISO-8859-1")

data = pd.concat([data1, data2, data3])

In [34]:
data = data.reset_index(drop=True)
data.groupby(1)[0].nunique()

# Quans Data

## Read in train and test txt files

In [2]:
train_genres = open("train_genres.txt", encoding="utf-8").read().split('\n')
train_plots = open("train_plots.txt", encoding="utf-8").read().split('\n')

In [3]:
train_data = pd.DataFrame({"genres": train_genres, "plots": train_plots})

In [4]:
len(train_data)

204682

In [5]:
train_data.loc[4]["genres"].split(" ")

['Adventure', 'Animation', 'Comedy', '']

In [6]:
test_genres = open("test_genres.txt", encoding="utf-8").read().split('\n')
test_plots = open("test_plots.txt", encoding="utf-8").read().split('\n')

In [7]:
test_data = pd.DataFrame({"genres": test_genres, "plots": test_plots})

In [8]:
len(test_data)

51171

In [9]:
#Concat dataframes
full_data = pd.concat([train_data, test_data], ignore_index= True)

In [10]:
#Convert genres to list for easier analysis
def list_genres(row):
    return row["genres"].split(" ")[:-1]

In [11]:
full_data["list_genres"] = full_data.apply(lambda row: list_genres(row), axis = 1)

In [30]:
count_dict = defaultdict(int)

In [31]:
#Count num of movies per genre
def dict_count(row):
    global count_dict
    for genre in row["list_genres"]:
        count_dict[genre] += 1

In [32]:
#RUN ONLY ONCE!!!
count_val_series = full_data.apply(lambda row: dict_count(row), axis = 1)

In [38]:
for key,val in count_dict.items():
    print("{:0.2f}% of the movies are {}".format(100*val/len(full_data), key))

27.88% of the movies are Comedy
45.84% of the movies are Drama
2.37% of the movies are Western
7.00% of the movies are Adventure
6.21% of the movies are Animation
9.71% of the movies are Action
11.18% of the movies are Thriller
8.08% of the movies are Family
11.01% of the movies are Romance
5.89% of the movies are Fantasy
7.77% of the movies are Horror
4.77% of the movies are History
4.95% of the movies are Music
5.60% of the movies are Sci-Fi
2.85% of the movies are War
7.27% of the movies are Crime
2.23% of the movies are Musical
5.18% of the movies are Biography
5.40% of the movies are Mystery
2.37% of the movies are Sport
